<a href="https://colab.research.google.com/github/ufbfung/cpic/blob/main/ethnicitylookup_noGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests

def get_drugid():

  # Prompt user for drug of choice
  drug = input("Enter name of drug, all lowercase: ")
  url = "https://api.cpicpgx.org/v1/drug?name=eq." + drug
  response = requests.get(url)
  while response.text == '[]' or response.status_code != 200:
    print("Invalid drug name entered. Please try again.")
    drug = input("Enter name of drug: ")
    url = "https://api.cpicpgx.org/v1/drug?name=eq." + drug
    response = requests.get(url)

  # Convert drug name to RxNorm drugID
  data = response.json()
  drugid = data[0]['drugid']
  return drugid

In [35]:
def get_genotype():

  # Prompt user for genotype
  genotype = input("Please enter patient genotype for CYP2C19 (e.g. *17/*17), press Enter if unknown genotype: ")
  return genotype


def get_ethnicity():

  # Prompt user for ethnicity
  choices=['Latino', 'American', 'European', 'Oceanian', 'East Asian',
          'Near Eastern', 'Central/South Asian', 'Sub-Saharan African',
          'African American/Afro-Caribbean', 'Other', 'Mixed Ethnicity',
          'Unknown']
  print("Please select one of the choices listed below: ")
  for c in choices:
    print(c)
  ethnicity = input("Ethnicity: ")
  while ethnicity not in choices:
    print("Invalid choice, please select again.")
    ethnicity = input("Ethnicity: ")
  return ethnicity

In [90]:
def get_lookupkey_from_genotype(genotype):

  url = "https://api.cpicpgx.org/v1/diplotype?genesymbol=eq.CYP2C19&diplotype=eq." + genotype
  response = requests.get(url)
  if response.status_code == 200 and response.text != '[]':
    data = response.json()
    lookupkey = data[0]['lookupkey']
    return lookupkey
  return None

In [80]:
def lookupkey_to_prob_list(ethnicity):

  url = "https://api.cpicpgx.org/v1/gene_result?genesymbol=eq.CYP2C19&frequency=not.is.null&order=result&select=result,frequency"
  response = requests.get(url)
  lookupkey_to_prob = []
  data = response.json()
  for item in data:
    lookupkey = {'CYP2C19': item['result']} 
    prob = item['frequency'][ethnicity]
    lookupkey_to_prob.append((lookupkey, prob))
  return lookupkey_to_prob

In [110]:
def get_recommendations_for_ethnicity(drugid):

  ethnicity = get_ethnicity()

  # User did not enter specific genotype or ethnicity
  if ethnicity == 'Unknown':
    print("Enquire about patient ethnicity, or recommend genetic testing for most precise results.")
    return None

  # Mixed ethnicity - no good way to handle this case
  if ethnicity == 'Mixed Ethnicity' or ethnicity == 'Other':
    print("Unfortunately our database does not currently contain data for this ethnicity.")
    print("We recommend genetic testing for most precise results.")
    return None

  # Valid ethnicity entered - now, get list of tuples (lookupkey, probability) for that specific ethnicity
  lookupkey_to_prob = lookupkey_to_prob_list(ethnicity)

  # Iterate through list and get recommendations
  recommendations = []
  for lookupkey, prob in sorted(lookupkey_to_prob, key = lambda k:k[1], reverse = True):

    # Set phenotype probability cutoff as desired for final output, or just output all phenotypes
    #if prob > 0.01:
    recommendations = recommendations + get_recommendation(drugid, json.dumps(lookupkey), prob)

  return recommendations


In [112]:
import requests
from tabulate import tabulate
import json

# Function that will retrieve the CPIC recommendation given a drugid and lookupkey as inputs

def get_recommendation(drugid, lookupkey, prob=1.0):
    url = f"https://api.cpicpgx.org/v1/recommendation?select=drug(name), guideline(name),* &drugid=eq.{drugid}&lookupkey=cs.{lookupkey}"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        recommendations = [{'Drug': item['drug']['name'],
                            'CYP2C19 Phenotype': lookupkey.split("\"")[3],
                            'Probability of Phenotype': prob, 
                            'Recommendation': item['drugrecommendation'],
                            'Guideline': item['guideline']['name']} for item in data]
        return recommendations
    return None

# Displays the drug, CYP2C19 phenotype, associated probability, recommendation, and guideline using the get_recommendation function.

def main():

    # Prompt user for drug name, and retrieve RxNorm drug ID
    drugid = get_drugid()

    # For now, hardcode gene as CYP2C19 - can modify this later to let user select a gene(s), or mention in future directions

    # Prompt user to enter patient genotype for CYP2C19/ethnicity
    genotype = get_genotype()
    lookupkey = get_lookupkey_from_genotype(genotype)
    by_ethnicity = False

    # If lookupkey is None, keep prompting user until they enter a valid genotype or unknown genotype
    while lookupkey is None:

      # If genotype is not known, get recommendations for all phenotypes, sorted by probability for patient's ethnicity
      if not genotype:
        recommendations = get_recommendations_for_ethnicity(drugid)
        by_ethnicity = True
        break

      # If invalid genotype entered, prompt user to try again
      print("Invalid CYP2C19 genotype entered, please try again.")
      genotype = get_genotype()
      lookupkey = get_lookupkey_from_genotype(genotype)

    
    # Known CYP2C19 genotype - find lookup key and use it to get recommendations
    if not by_ethnicity:
        recommendations = get_recommendation(drugid, json.dumps(lookupkey))  # json.dumps to convert '' Python strings to "" JSON...

    if recommendations:
        headers = {'Drug': 'Drug', 'CYP2C19 Phenotype':'CYP2C19 Phenotype', 'Probability of Phenotype': 'Probability of Phenotype: ', 
                   'Recommendation': 'Recommendation', 'Guideline': 'Guideline'}
        print('CPIC Recommendations:')
        print(tabulate(recommendations, headers=headers))
    else:
        print("No recommendations found.")

if __name__ == '__main__':
    main()


Enter name of drug, all lowercase: sertraline
Please enter patient genotype for CYP2C19 (e.g. *17/*17), press Enter if unknown genotype: 
Please select one of the choices listed below: 
Latino
American
European
Oceanian
East Asian
Near Eastern
Central/South Asian
Sub-Saharan African
African American/Afro-Caribbean
Other
Mixed Ethnicity
Unknown
Ethnicity: Latino
CPIC Recommendations:
Drug        CYP2C19 Phenotype                  Probability of Phenotype:   Recommendation                                                                                                                                                                       Guideline
----------  -------------------------------  ----------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------------------------------------
sertraline  Normal Metabolizer  

In [56]:
import requests
from prettytable import PrettyTable
from tabulate import tabulate

# Function that retrieves the lookupkeys (aka the phenotypes) for a gene of interest.
# Note that lookupkeys may not always be the same, depending on the gene.
# Plan will be to implement only a subset of genes so we will have more control over this.

def get_lookupkeys(gene_symbol):
    url = f"https://api.cpicpgx.org/v1/diplotype?genesymbol=eq.{gene_symbol}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        unique_lookupkeys = []
        for item in data:
            lookupkey = item['lookupkey']
            if lookupkey not in unique_lookupkeys:
                unique_lookupkeys.append(lookupkey)
        return unique_lookupkeys
    return None

# Function that will display the lookupkeys in a nice table format

def print_table(headers, rows):
    table = PrettyTable(headers)
    for row in rows:
        table.add_row(row)
    print(table)

In [53]:
# This will allow a user to enter a gene symbol (aka gene of interest) and call the get_lookupkeys function
# to retrieve a table of lookupkeys

gene_symbol = input("Enter gene symbol: ")
lookupkeys = get_lookupkeys(gene_symbol)

if lookupkeys:
    headers = ['Lookupkeys']
    rows = [[lookupkey] for lookupkey in lookupkeys]
    print_table(headers, rows)
else:
    print(f"No lookupkeys found for {gene_symbol}.")

Enter gene symbol: CYP2C19
+------------------------------------------------+
|                   Lookupkeys                   |
+------------------------------------------------+
|     {'CYP2C19': 'Ultrarapid Metabolizer'}      |
|        {'CYP2C19': 'Rapid Metabolizer'}        |
|       {'CYP2C19': 'Normal Metabolizer'}        |
| {'CYP2C19': 'Likely Intermediate Metabolizer'} |
|    {'CYP2C19': 'Intermediate Metabolizer'}     |
|     {'CYP2C19': 'Likely Poor Metabolizer'}     |
|        {'CYP2C19': 'Poor Metabolizer'}         |
|          {'CYP2C19': 'Indeterminate'}          |
+------------------------------------------------+
